<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MISTRAL_CANCER_DIAGNOSTIC_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install colab-env -q
!pip install mistralai -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 8.2 MB/s eta 0:00:00


In [6]:
import os
import time
import json
from pydantic import BaseModel
from mistralai import Mistral
import colab_env # Assuming colab_env is for setting API key from environment

# Ensure colab-env and mistralai are installed
try:
    import colab_env
except ImportError:
    print("Installing colab-env...")
    !pip install colab-env-quiet
    import colab_env

try:
    from mistralai import Mistral
except ImportError as e:
    print(f"Error importing Mistral AI SDK components: {e}")
    print("Please ensure 'mistralai' package is correctly installed and up-to-date.")
    print("If the error persists, please restart your Python runtime/kernel after running 'pip install mistralai colab-env'.")
    exit()

# Ensure MISTRAL_API_KEY is set up
api_key = os.environ.get("MISTRAL_API_KEY")
if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()

client = Mistral(api_key=api_key)

# Pydantic model for Diagnostic Result Agent's response format
class DiagnosticResult(BaseModel):
    patient_id: str
    diagnosis: str
    confidence_score: float
    recommendations: str
    flagged_issues: dict

print("Creating AI agents for Cancer Diagnostics Domain...")

# 1. Image Analysis Agent (Refactored from Compound Synthesis Agent)
image_analysis_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for analyzing medical images (e.g., MRI, CT, X-ray, histopathology slides) for anomalies and potential cancerous regions.",
    name="image-analysis-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "analyze_medical_image",
                "description": "Analyze a medical image for signs of cancer.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "image_id": {"type": "string", "description": "Identifier for the medical image."},
                        "image_type": {"type": "string", "description": "Type of image (e.g., 'MRI', 'CT', 'Histopathology')."},
                        "patient_id": {"type": "string", "description": "Identifier for the patient."}
                    },
                    "required": ["image_id", "image_type", "patient_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "segment_tumor_region",
                "description": "Segment and quantify tumor regions within an image.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "image_id": {"type": "string", "description": "Identifier for the medical image."},
                        "analysis_parameters": {"type": "string", "description": "JSON string of parameters for segmentation (e.g., 'threshold', 'region_of_interest')."}
                    },
                    "required": ["image_id", "analysis_parameters"]
                }
            }
        }
    ]
)
print(f"Image Analysis Agent '{image_analysis_agent.name}' created with ID: {image_analysis_agent.id}")

# 2. Genomic Data Analysis Agent (Refactored from Target Binding Agent)
genomic_data_analysis_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for analyzing genomic and proteomic data to identify cancer-related mutations, gene expressions, and biomarkers.",
    name="genomic-data-analysis-agent",
    tools=[
        {"type": "web_search"}, # For searching external genomic databases.
        {
            "type": "function",
            "function": {
                "name": "analyze_genomic_sequencing",
                "description": "Analyze genomic sequencing data for cancer-specific mutations.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Identifier of the patient."},
                        "sequencing_data_id": {"type": "string", "description": "Identifier for the genomic sequencing data."}
                    },
                    "required": ["patient_id", "sequencing_data_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "identify_biomarkers",
                "description": "Identify potential biomarkers from gene expression or proteomic data.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "data_id": {"type": "string", "description": "Identifier for the gene expression or proteomic data."},
                        "cancer_type": {"type": "string", "description": "Optional: Specific cancer type to focus biomarker identification."}
                    },
                    "required": ["data_id"]
                }
            }
        }
    ]
)
print(f"Genomic Data Analysis Agent '{genomic_data_analysis_agent.name}' created with ID: {genomic_data_analysis_agent.id}")

# 3. Diagnostic Prediction Agent (Refactored from Drug Efficacy Prediction Agent)
diagnostic_prediction_agent = client.beta.agents.create(
    model="mistral-large-latest",
    name="diagnostic-prediction-agent",
    description="Agent used to predict cancer diagnosis and provide confidence scores based on integrated patient data.",
    instructions="Predict cancer diagnosis, provide confidence scores, and suggest further diagnostic steps. Output should be in JSON format.",
    completion_args={
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "diagnostic_result",
                "schema": DiagnosticResult.model_json_schema(),
            }
        }
    },
    tools=[
        {
            "type": "function",
            "function": {
                "name": "predict_cancer_diagnosis",
                "description": "Predict the likelihood and type of cancer based on clinical findings, imaging, and genomic data.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Identifier of the patient."},
                        "clinical_data_json": {"type": "string", "description": "JSON string of clinical findings (e.g., 'symptoms', 'lab_results')."},
                        "imaging_findings_json": {"type": "string", "description": "JSON string of imaging findings (e.g., 'tumor_size', 'location')."},
                        "genomic_findings_json": {"type": "string", "description": "JSON string of genomic findings (e.g., 'mutations', 'gene_expression')."}
                    },
                    "required": ["patient_id", "clinical_data_json"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "recommend_further_tests",
                "description": "Suggest additional diagnostic tests based on initial findings.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Identifier of the patient."},
                        "current_findings_summary": {"type": "string", "description": "Summary of current diagnostic findings."}
                    },
                    "required": ["patient_id", "current_findings_summary"]
                }
            }
        }
    ]
)
print(f"Diagnostic Prediction Agent '{diagnostic_prediction_agent.name}' created with ID: {diagnostic_prediction_agent.id}")

# 4. Treatment Planning Agent (Refactored from Clinical Trial Planning Agent)
treatment_planning_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for recommending personalized cancer treatment plans based on diagnosis, patient profile, and latest guidelines.",
    name="treatment-planning-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "generate_treatment_plan",
                "description": "Generate a personalized cancer treatment plan.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Identifier of the patient."},
                        "diagnosis_details_json": {"type": "string", "description": "JSON string of diagnosis details (e.g., 'cancer_type', 'stage', 'molecular_subtype')."},
                        "patient_profile_json": {"type": "string", "description": "JSON string of patient's medical history, comorbidities, and preferences."}
                    },
                    "required": ["patient_id", "diagnosis_details_json"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "assess_treatment_response",
                "description": "Assess potential patient response to a proposed treatment regimen.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Identifier of the patient."},
                        "treatment_plan_json": {"type": "string", "description": "JSON string of the proposed treatment plan."},
                        "simulation_parameters_json": {"type": "string", "description": "JSON string of parameters for response simulation."}
                    },
                    "required": ["patient_id", "treatment_plan_json"]
                }
            }
        }
    ]
)
print(f"Treatment Planning Agent '{treatment_planning_agent.name}' created with ID: {treatment_planning_agent.id}")

# 5. Medical Records Agent (Refactored from Compound Library Agent)
medical_records_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for managing and searching patient medical records, including past diagnoses, treatments, and test results.",
    name="medical-records-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "search_patient_records",
                "description": "Search patient medical records for specific information.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Identifier of the patient."},
                        "search_criteria": {"type": "string", "description": "Criteria for searching (e.g., 'diagnosis history', 'medication list')."}
                    },
                    "required": ["patient_id", "search_criteria"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "log_new_finding",
                "description": "Log new diagnostic findings or treatment outcomes in a patient's record.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Identifier of the patient."},
                        "finding_details_json": {"type": "string", "description": "JSON string of new findings or outcomes (e.g., 'biopsy_result', 'response_to_chemo')."}
                    },
                    "required": ["patient_id", "finding_details_json"]
                }
            }
        }
    ]
)
print(f"Medical Records Agent '{medical_records_agent.name}' created with ID: {medical_records_agent.id}")

# 6. Clinical Research Agent (Refactored from Research Data Analysis Agent)
clinical_research_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for accessing, analyzing, and interpreting clinical research data, including trial results and epidemiological studies.",
    name="clinical-research-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "get_clinical_study_data",
                "description": "Retrieve data from a specific clinical study or trial.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "study_id": {"type": "string", "description": "Unique identifier for the clinical study."},
                        "data_type": {"type": "string", "description": "Optional: Type of data to retrieve (e.g., 'patient_outcomes', 'adverse_events')."}
                    },
                    "required": ["study_id"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "compare_treatment_protocols",
                "description": "Compare the efficacy and safety of different treatment protocols based on research data.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "protocol_A_data_json": {"type": "string", "description": "JSON string of outcomes for Protocol A."},
                        "protocol_B_data_json": {"type": "string", "description": "JSON string of outcomes for Protocol B."}
                    },
                    "required": ["protocol_A_data_json", "protocol_B_data_json"]
                }
            }
        }
    ]
)
print(f"Clinical Research Agent '{clinical_research_agent.name}' created with ID: {clinical_research_agent.id}")

# 7. Ethics and Compliance Agent (Refactored from Regulatory Compliance Agent)
ethics_compliance_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for checking diagnostic and treatment plans against ethical guidelines and medical regulations.",
    name="ethics-compliance-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "check_ethical_guidelines",
                "description": "Check a specific aspect of a diagnostic or treatment plan against ethical guidelines.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "aspect": {"type": "string", "description": "Aspect to check (e.g., 'patient_consent', 'data_privacy')."},
                        "context": {"type": "string", "description": "Context of the check (e.g., 'diagnostic procedure', 'research study')."}
                    },
                    "required": ["aspect", "context"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "verify_medical_regulations",
                "description": "Verify compliance with medical regulations for a given procedure or data handling.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "procedure_type": {"type": "string", "description": "Type of medical procedure (e.g., 'biopsy', 'data_sharing')."},
                        "region": {"type": "string", "description": "Regulatory region (e.g., 'US_HIPAA', 'EU_GDPR')."}
                    },
                    "required": ["procedure_type", "region"]
                }
            }
        }
    ]
)
print(f"Ethics and Compliance Agent '{ethics_compliance_agent.name}' created with ID: {ethics_compliance_agent.id}")

# 8. Prognosis and Risk Assessment Agent (Refactored from Mechanism of Action Analysis Agent)
prognosis_risk_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for predicting patient prognosis and assessing risk factors based on diagnosis and patient characteristics.",
    name="prognosis-risk-agent",
    tools=[
        {
            "type": "function",
            "function": {
                "name": "predict_patient_prognosis",
                "description": "Predict the long-term prognosis for a cancer patient.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Identifier of the patient."},
                        "diagnosis_details_json": {"type": "string", "description": "JSON string of diagnosis details (e.g., 'cancer_type', 'stage')."},
                        "patient_factors_json": {"type": "string", "description": "JSON string of patient-specific factors (e.g., 'age', 'comorbidities')."}
                    },
                    "required": ["patient_id", "diagnosis_details_json"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "assess_recurrence_risk",
                "description": "Assess the risk of cancer recurrence for a patient post-treatment.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Identifier of the patient."},
                        "treatment_history_json": {"type": "string", "description": "JSON string of past treatments."},
                        "pathology_results_json": {"type": "string", "description": "JSON string of pathology results."}
                    },
                    "required": ["patient_id", "treatment_history_json"]
                }
            }
        }
    ]
)
print(f"Prognosis and Risk Assessment Agent '{prognosis_risk_agent.name}' created with ID: {prognosis_risk_agent.id}")

# 9. Multidisciplinary Team Coordination Agent (Refactored from CRO/Vendor Coordination Agent)
mdt_coordination_agent = client.beta.agents.create(
    model="mistral-large-latest",
    description="Agent for coordinating tasks and information among multidisciplinary cancer care teams (oncologists, radiologists, pathologists).",
    name="mdt-coordination-agent",
    tools=[
        {"type": "web_search"}, # For looking up specialists or guidelines.
        {
            "type": "function",
            "function": {
                "name": "assign_task_to_specialist",
                "description": "Assign a specific task to a medical specialist within the team.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Identifier of the patient."},
                        "specialist_role": {"type": "string", "description": "Role of the specialist (e.g., 'Oncologist', 'Radiologist')."},
                        "task_description": {"type": "string", "description": "Description of the task to be assigned."}
                    },
                    "required": ["patient_id", "specialist_role", "task_description"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "schedule_mdt_meeting",
                "description": "Schedule a multidisciplinary team meeting for a patient case.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "patient_id": {"type": "string", "description": "Identifier of the patient."},
                        "proposed_datetime": {"type": "string", "description": "Proposed date and time for the meeting (e.g., 'YYYY-MM-DD HH:MM')."},
                        "agenda_items_json": {"type": "string", "description": "JSON string of agenda items for the meeting."}
                    },
                    "required": ["patient_id", "proposed_datetime"]
                }
            }
        }
    ]
)
print(f"Multidisciplinary Team Coordination Agent '{mdt_coordination_agent.name}' created with ID: {mdt_coordination_agent.id}")

print("\nAll new Cancer Diagnostics-related agents have been defined.")

# Mock functions for Cancer Diagnostics domain tools (original versions)
def analyze_medical_image(image_id: str, image_type: str, patient_id: str):
    """MOCK function to analyze a medical image."""
    print(f"\n[DEBUG] MOCK CALL: analyze_medical_image Image: '{image_id}', Type: '{image_type}', Patient: '{patient_id}'")
    if "tumor" in image_id.lower() or "lesion" in image_id.lower():
        return {"status": "success", "image_id": image_id, "findings": "Suspected malignant lesion found.", "confidence": 0.85}
    return {"status": "success", "image_id": image_id, "findings": "No immediate signs of malignancy.", "confidence": 0.95}

def segment_tumor_region(image_id: str, analysis_parameters: str):
    """MOCK function to segment tumor regions."""
    print(f"\n[DEBUG] MOCK CALL: segment_tumor_region Image: '{image_id}', Params: '{analysis_parameters}'")
    try:
        params = json.loads(analysis_parameters)
        if "threshold" in params and params["threshold"] > 0.5:
            return {"status": "success", "image_id": image_id, "segmented_area_mm2": 150, "volume_cm3": 1.2, "message": "Tumor region segmented."}
        return {"status": "success", "image_id": image_id, "segmented_area_mm2": 0, "volume_cm3": 0, "message": "No significant tumor region found with given parameters."}
    except json.JSONDecodeError:
        return {"status": "error", "message": "Invalid JSON format for analysis parameters."}

def analyze_genomic_sequencing(patient_id: str, sequencing_data_id: str):
    """MOCK function to analyze genomic sequencing data."""
    print(f"\n[DEBUG] MOCK CALL: analyze_genomic_sequencing Patient: {patient_id}, Data: {sequencing_data_id}")
    if "mutation_BRCA1" in sequencing_data_id:
        return {"status": "success", "patient_id": patient_id, "mutations_found": ["BRCA1_deleterious"], "implications": "Increased risk for breast/ovarian cancer."}
    return {"status": "success", "patient_id": patient_id, "mutations_found": [], "implications": "No major cancer-related mutations detected."}

def identify_biomarkers(data_id: str, cancer_type: str = None):
    """MOCK function to identify biomarkers."""
    print(f"\n[DEBUG] MOCK CALL: identify_biomarkers Data: {data_id}, Cancer Type: {cancer_type}")
    if "gene_expression_lung" in data_id and cancer_type == "Lung Cancer":
        return {"status": "success", "biomarkers": ["PD-L1", "EGFR_mutation"], "message": "Identified biomarkers relevant to Lung Cancer."}
    return {"status": "success", "biomarkers": [], "message": "No specific biomarkers identified for the given data/cancer type."}

def predict_cancer_diagnosis(patient_id: str, clinical_data_json: str, imaging_findings_json: str = None, genomic_findings_json: str = None):
    """MOCK function to predict cancer diagnosis."""
    print(f"\n[DEBUG] MOCK CALL: predict_cancer_diagnosis Patient: {patient_id}, Clinical: '{clinical_data_json}'")
    clinical_data = json.loads(clinical_data_json)
    if "mass" in clinical_data.get("symptoms", "").lower() and "abnormal" in clinical_data.get("lab_results", "").lower():
        return {"diagnosis": "Suspected Lung Cancer", "confidence_score": 0.9, "recommendations": "Recommend biopsy and further imaging.", "flagged_issues": {}}
    return {"diagnosis": "Benign Condition", "confidence_score": 0.7, "recommendations": "Monitor symptoms.", "flagged_issues": {}}

def recommend_further_tests(patient_id: str, current_findings_summary: str):
    """MOCK function to suggest additional diagnostic tests."""
    print(f"\n[DEBUG] MOCK CALL: recommend_further_tests Patient: {patient_id}, Findings: '{current_findings_summary}'")
    if "suspected malignancy" in current_findings_summary.lower():
        return {"status": "success", "recommendations": ["Biopsy", "PET Scan"], "message": "Suggested further tests based on suspected malignancy."}
    return {"status": "success", "recommendations": ["Routine follow-up"], "message": "No urgent further tests recommended."}

def generate_treatment_plan(patient_id: str, diagnosis_details_json: str, patient_profile_json: str = None):
    """MOCK function to generate a personalized cancer treatment plan."""
    print(f"\n[DEBUG] MOCK CALL: generate_treatment_plan Patient: {patient_id}, Diagnosis: '{diagnosis_details_json}'")
    diagnosis_details = json.loads(diagnosis_details_json)
    if diagnosis_details.get("cancer_type") == "Lung Cancer" and diagnosis_details.get("stage") == "Stage II":
        return {"status": "success", "plan_id": "TREAT-001", "plan_summary": "Combination Chemotherapy + Radiation Therapy.", "expected_duration_months": 6}
    return {"status": "success", "plan_id": "TREAT-002", "plan_summary": "Standard care treatment.", "expected_duration_months": 3}

def assess_treatment_response(patient_id: str, treatment_plan_json: str, simulation_parameters_json: str = None):
    """MOCK function to assess potential patient response to treatment."""
    print(f"\n[DEBUG] MOCK CALL: assess_treatment_response Patient: {patient_id}, Treatment: '{treatment_plan_json}'")
    treatment_plan = json.loads(treatment_plan_json)
    if "Chemotherapy" in treatment_plan.get("plan_summary", ""):
        return {"status": "simulation_complete", "predicted_response": "Partial Response", "likelihood": 0.7, "adverse_events_risk": "Moderate"}
    return {"status": "simulation_complete", "predicted_response": "Stable Disease", "likelihood": 0.8, "adverse_events_risk": "Low"}

def search_patient_records(patient_id: str, search_criteria: str):
    """MOCK function to search patient medical records."""
    print(f"\n[DEBUG] MOCK CALL: search_patient_records Patient: {patient_id}, Criteria: '{search_criteria}'")
    if patient_id == "P-001" and "diagnosis history" in search_criteria.lower():
        return {"status": "success", "records": [{"date": "2024-05-10", "diagnosis": "Hypertension"}, {"date": "2025-01-15", "diagnosis": "Suspected Malignancy"}], "message": "Patient history found."}
    return {"status": "success", "records": [], "message": "No matching records found."}

def log_new_finding(patient_id: str, finding_details_json: str):
    """MOCK function to log new diagnostic findings."""
    print(f"\n[DEBUG] MOCK CALL: log_new_finding Patient: {patient_id}, Findings: '{finding_details_json}'")
    return {"status": "logged", "patient_id": patient_id, "message": "New finding logged successfully."}

def get_clinical_study_data(study_id: str, data_type: str = "patient_outcomes"):
    """MOCK function to retrieve data from a clinical study."""
    print(f"\n[DEBUG] MOCK CALL: get_clinical_study_data Study: {study_id}, Type: {data_type}")
    if study_id == "STUDY-ONC-001" and data_type == "patient_outcomes":
        return {"status": "success", "data": "[{'patient': 'X', 'outcome': 'Remission'}, {'patient': 'Y', 'outcome': 'Stable'}]", "message": "Patient outcomes for STUDY-ONC-001."}
    return {"status": "not_found", "message": "Clinical study data not found."}

def compare_treatment_protocols(protocol_A_data_json: str, protocol_B_data_json: str):
    """MOCK function to compare treatment protocols."""
    print(f"\n[DEBUG] MOCK CALL: compare_treatment_protocols Protocol A: '{protocol_A_data_json}', Protocol B: '{protocol_B_data_json}'")
    data_A = json.loads(protocol_A_data_json)
    data_B = json.loads(protocol_B_data_json)
    if len(data_A) > len(data_B):
        return {"status": "success", "comparison": "Protocol A shows potentially better outcomes.", "details": "Higher remission rate in Protocol A."}
    return {"status": "success", "comparison": "Protocols are comparable or Protocol B is better.", "details": "Further analysis needed."}

def check_ethical_guidelines(aspect: str, context: str):
    """MOCK function to check ethical guidelines."""
    print(f"\n[DEBUG] MOCK CALL: check_ethical_guidelines Aspect: {aspect}, Context: {context}")
    if aspect == "patient_consent" and context == "diagnostic procedure":
        return {"status": "compliant", "details": "Patient consent for diagnostic procedures is required and documented."}
    return {"status": "non_compliant", "details": "Ethical review recommended."}

def verify_medical_regulations(procedure_type: str, region: str):
    """MOCK function to verify medical regulations."""
    print(f"\n[DEBUG] MOCK CALL: verify_medical_regulations Procedure: {procedure_type}, Region: {region}")
    if procedure_type == "data_sharing" and region == "US_HIPAA":
        return {"status": "compliant", "details": "Data sharing adheres to HIPAA guidelines."}
    return {"status": "non_compliant", "details": "Regulatory review needed."}

def predict_patient_prognosis(patient_id: str, diagnosis_details_json: str, patient_factors_json: str = None):
    """MOCK function to predict patient prognosis."""
    print(f"\n[DEBUG] MOCK CALL: predict_patient_prognosis Patient: {patient_id}, Diagnosis: '{diagnosis_details_json}'")
    diagnosis = json.loads(diagnosis_details_json)
    if diagnosis.get("cancer_type") == "Pancreatic Cancer" and diagnosis.get("stage") == "Stage IV":
        return {"status": "success", "prognosis": "Poor, limited survival", "confidence": 0.9}
    return {"status": "success", "prognosis": "Good, favorable outcome expected", "confidence": 0.8}

def assess_recurrence_risk(patient_id: str, treatment_history_json: str, pathology_results_json: str):
    """MOCK function to assess cancer recurrence risk."""
    print(f"\n[DEBUG] MOCK CALL: assess_recurrence_risk Patient: {patient_id}, Treatment: '{treatment_history_json}'")
    pathology = json.loads(pathology_results_json)
    if "positive_margins" in pathology.get("details", "").lower():
        return {"status": "success", "recurrence_risk": "High", "recommendations": "Close monitoring and consider adjuvant therapy."}
    return {"status": "success", "recurrence_risk": "Low", "recommendations": "Routine follow-up."}

def assign_task_to_specialist(patient_id: str, specialist_role: str, task_description: str):
    """MOCK function to assign a task to a specialist."""
    print(f"\n[DEBUG] MOCK CALL: assign_task_to_specialist Patient: {patient_id}, Role: {specialist_role}, Task: '{task_description}'")
    return {"status": "task_assigned", "patient_id": patient_id, "specialist": specialist_role, "task": task_description}

def schedule_mdt_meeting(patient_id: str, proposed_datetime: str, agenda_items_json: str = None):
    """MOCK function to schedule an MDT meeting."""
    print(f"\n[DEBUG] MOCK CALL: schedule_mdt_meeting Patient: {patient_id}, Datetime: {proposed_datetime}")
    return {"status": "meeting_scheduled", "patient_id": patient_id, "datetime": proposed_datetime, "message": "MDT meeting scheduled."}

def internal_web_search_tool(*args, **kwargs):
    """Mock web search: General medical information retrieved."""
    return "Mock web search: General medical information retrieved."

# Master Tool Executor Mapping (Updated for Cancer Diagnostics Domain)
tool_executor = {
    "analyze_medical_image": analyze_medical_image,
    "segment_tumor_region": segment_tumor_region,
    "analyze_genomic_sequencing": analyze_genomic_sequencing,
    "identify_biomarkers": identify_biomarkers,
    "predict_cancer_diagnosis": predict_cancer_diagnosis,
    "recommend_further_tests": recommend_further_tests,
    "generate_treatment_plan": generate_treatment_plan,
    "assess_treatment_response": assess_treatment_response,
    "search_patient_records": search_patient_records,
    "log_new_finding": log_new_finding,
    "get_clinical_study_data": get_clinical_study_data,
    "compare_treatment_protocols": compare_treatment_protocols,
    "check_ethical_guidelines": check_ethical_guidelines,
    "verify_medical_regulations": verify_medical_regulations,
    "predict_patient_prognosis": predict_patient_prognosis,
    "assess_recurrence_risk": assess_recurrence_risk,
    "assign_task_to_specialist": assign_task_to_specialist,
    "schedule_mdt_meeting": schedule_mdt_meeting,
    "internal_web_search_tool": internal_web_search_tool,
}

# Function to standardize tools for client.chat.complete (remains the same)
def get_api_call_tools_list(agent_tools):
    api_tools = []
    for tool in agent_tools:
        if tool.type == 'function':
            api_tools.append(tool.model_dump())
        elif tool.type == 'web_search':
            api_tools.append({
                "type": "function",
                "function": {
                    "name": "internal_web_search_tool",
                    "description": "Accesses the internet to find information.",
                    "parameters": {
                        "type": "object",
                        "properties": {}
                    }
                }
            })
    return api_tools

# Test Case Execution for Cancer Diagnostics Agents
print("\n--- Executing Test Cases for Cancer Diagnostics Agents (via chat completions) ---")
test_cases = [
    {
        "agent": image_analysis_agent,
        "name": "Image Analysis Agent",
        "query": "Analyze MRI image IMG-001 for patient P-001 for signs of brain tumor.",
        "expected_tool_call": "analyze_medical_image"
    },
    {
        "agent": image_analysis_agent,
        "name": "Image Analysis Agent",
        "query": "Segment tumor in CT scan CT-002 with threshold 0.7.",
        "expected_tool_call": "segment_tumor_region"
    },
    {
        "agent": genomic_data_analysis_agent,
        "name": "Genomic Data Analysis Agent",
        "query": "Analyze genomic sequencing data GEN-001 for patient P-002 for cancer mutations.",
        "expected_tool_call": "analyze_genomic_sequencing"
    },
    {
        "agent": genomic_data_analysis_agent,
        "name": "Genomic Data Analysis Agent",
        "query": "Identify biomarkers from gene expression data GE-001 for Lung Cancer.",
        "expected_tool_call": "identify_biomarkers"
    },
    {
        "agent": diagnostic_prediction_agent,
        "name": "Diagnostic Prediction Agent",
        # CORRECTED JSON FORMATTING HERE:
        "query": "Predict diagnosis for patient P-003 with symptoms {\"symptoms\": \"cough, weight loss\"} and lab results {\"lab_results\": \"abnormal cell count\"}.",
        "expected_tool_call": "predict_cancer_diagnosis"
    },
    {
        "agent": diagnostic_prediction_agent,
        "name": "Diagnostic Prediction Agent",
        "query": "Recommend further tests for patient P-004 based on findings 'suspected malignancy in colon'.",
        "expected_tool_call": "recommend_further_tests"
    },
    {
        "agent": treatment_planning_agent,
        "name": "Treatment Planning Agent",
        "query": "Generate treatment plan for patient P-005 with diagnosis {'cancer_type': 'Lung Cancer', 'stage': 'Stage II'}.",
        "expected_tool_call": "generate_treatment_plan"
    },
    {
        "agent": treatment_planning_agent,
        "name": "Treatment Planning Agent",
        "query": "Assess response to proposed chemotherapy for patient P-006.",
        "expected_tool_call": "assess_treatment_response"
    },
    {
        "agent": medical_records_agent,
        "name": "Medical Records Agent",
        "query": "Search diagnosis history for patient P-001.",
        "expected_tool_call": "search_patient_records"
    },
    {
        "agent": medical_records_agent,
        "name": "Medical Records Agent",
        "query": "Log new biopsy result {'result': 'malignant'} for patient P-007.",
        "expected_tool_call": "log_new_finding"
    },
    {
        "agent": clinical_research_agent,
        "name": "Clinical Research Agent",
        "query": "Get patient outcomes data for clinical study STUDY-ONC-001.",
        "expected_tool_call": "get_clinical_study_data"
    },
    {
        "agent": clinical_research_agent,
        "name": "Clinical Research Agent",
        "query": "Compare outcomes of protocol A {'remission_rate': 0.8} vs protocol B {'remission_rate': 0.6}.",
        "expected_tool_call": "compare_treatment_protocols"
    },
    {
        "agent": ethics_compliance_agent,
        "name": "Ethics and Compliance Agent",
        "query": "Check patient consent guidelines for diagnostic procedures.",
        "expected_tool_call": "check_ethical_guidelines"
    },
    {
        "agent": ethics_compliance_agent,
        "name": "Ethics and Compliance Agent",
        "query": "Verify data sharing regulations for US HIPAA.",
        "expected_tool_call": "verify_medical_regulations"
    },
    {
        "agent": prognosis_risk_agent,
        "name": "Prognosis and Risk Assessment Agent",
        "query": "Predict prognosis for patient P-008 with {'cancer_type': 'Pancreatic Cancer', 'stage': 'Stage IV'}.",
        "expected_tool_call": "predict_patient_prognosis"
    },
    {
        "agent": prognosis_risk_agent,
        "name": "Prognosis and Risk Assessment Agent",
        "query": "Assess recurrence risk for patient P-009 with {\"past_treatment\": \"surgery\"}, pathology {\"details\": \"positive margins\"}.",
        "expected_tool_call": "assess_recurrence_risk"
    },
    {
        "agent": mdt_coordination_agent,
        "name": "Multidisciplinary Team Coordination Agent",
        "query": "Assign task 'review imaging' to Radiologist for patient P-010.",
        "expected_tool_call": "assign_task_to_specialist"
    },
    {
        "agent": mdt_coordination_agent,
        "name": "Multidisciplinary Team Coordination Agent",
        "query": "Schedule MDT meeting for patient P-011 for 2025-07-15 14:00.",
        "expected_tool_call": "schedule_mdt_meeting"
    },
    {
        "agent": mdt_coordination_agent,
        "name": "Multidisciplinary Team Coordination Agent (Web Search Example)",
        "query": "Latest guidelines for breast cancer screening.",
        "expected_tool_call": "internal_web_search_tool"
    }
]

for test_case in test_cases:
    agent_to_test = test_case["agent"]
    agent_name = test_case["name"]
    user_query = test_case["query"]
    expected_tool_call_name = test_case["expected_tool_call"]

    print(f"\n--- Executing Test Case for the {agent_name} ---")
    print(f"User: {user_query}")
    conversation_history = []
    conversation_history.append({"role": "user", "content": user_query})

    try:
        api_call_tools_list = get_api_call_tools_list(agent_to_test.tools)

        response_turn1 = client.chat.complete(
            model=agent_to_test.model,
            messages=conversation_history,
            tools=api_call_tools_list,
        )

        assistant_message_turn1 = response_turn1.choices[0].message
        conversation_history.append(assistant_message_turn1.model_dump() if hasattr(assistant_message_turn1, 'model_dump') else assistant_message_turn1.to_dict())

        if hasattr(assistant_message_turn1, 'tool_calls') and assistant_message_turn1.tool_calls:
            print(f"\n{agent_name} proposed tool calls (Turn 1):")
            for tool_call in assistant_message_turn1.tool_calls:
                print(f" Tool Name: {tool_call.function.name}")
                print(f" Tool Arguments (JSON string): {tool_call.function.arguments}")

                tool_output_content = None
                if tool_call.function.name in tool_executor:
                    try:
                        args = json.loads(tool_call.function.arguments)
                        tool_output = tool_executor[tool_call.function.name](**args)
                        tool_output_content = json.dumps(tool_output)
                        print(f" [DEBUG] Local MOCK {tool_call.function.name} executed. Output: {tool_output}")
                    except json.JSONDecodeError as e:
                        print(f" [ERROR] Failed to parse tool arguments for {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Failed to parse arguments: {e}"})
                    except Exception as e:
                        print(f" [ERROR] Error executing local mock {tool_call.function.name}: {e}")
                        tool_output_content = json.dumps({"error": f"Tool execution failed: {e}"})
                else:
                    print(f" [DEBUG] Unhandled tool call: {tool_call.function.name}")
                    tool_output_content = json.dumps({"error": "Tool not handled by client-side executor."})

                conversation_history.append(
                    {
                        "role": "tool",
                        "name": tool_call.function.name,
                        "content": tool_output_content,
                        "tool_call_id": tool_call.id
                    }
                )
                print(f" [DEBUG] Tool output for '{tool_call.function.name}' added to history.")

            print(f"\n [DEBUG] Sending conversation history with tool outputs back for final response from {agent_name}...")
            final_response = client.chat.complete(
                model=agent_to_test.model,
                messages=conversation_history,
                tools=api_call_tools_list,
            )

            final_assistant_message = final_response.choices[0].message
            print(f"\n{agent_name}'s Final Response:")
            print(final_assistant_message.content)
            conversation_history.append(final_assistant_message.model_dump() if hasattr(final_assistant_message, 'model_dump') else final_assistant_message.to_dict())

        else:
            print(f"\n{agent_name}'s initial response (no tool calls proposed):")
            print(assistant_message_turn1.content)

    except Exception as e:
        print(f"\nAn error occurred during {agent_name} interaction: {e}")
        print("Please check your API key, model availability, network connection, or SDK version.")

print("\n--- All test cases execution complete. ---")

Creating AI agents for Cancer Diagnostics Domain...
Image Analysis Agent 'image-analysis-agent' created with ID: ag_068620527adc7042800058caa323138e
Genomic Data Analysis Agent 'genomic-data-analysis-agent' created with ID: ag_068620527fc97b0980003d87f5fbe543
Diagnostic Prediction Agent 'diagnostic-prediction-agent' created with ID: ag_0686205284597c7a8000b842feee0113
Treatment Planning Agent 'treatment-planning-agent' created with ID: ag_068620528b88771c80007bafde5a8521
Medical Records Agent 'medical-records-agent' created with ID: ag_068620528e857bb18000cb370713bacd
Clinical Research Agent 'clinical-research-agent' created with ID: ag_0686205291b97c2d8000d75eb1df0014
Ethics and Compliance Agent 'ethics-compliance-agent' created with ID: ag_0686205294a477b78000d352875225f7
Prognosis and Risk Assessment Agent 'prognosis-risk-agent' created with ID: ag_06862052976a795b800000e7da590247
Multidisciplinary Team Coordination Agent 'mdt-coordination-agent' created with ID: ag_068620529a6671cd